In [1]:
# import your favourite libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('co2_emissions_kt_by_country.csv')

In [11]:
# check the data
display(df.sample(10))
print(df.info())

,country_code,country_name,year,value
10205,PRK,"Korea, Dem. People's Rep.",1975,107740.127000
4684,GNB,Guinea-Bissau,1998,170.000000
5017,GUY,Guyana,1974,1554.808000
6623,KIR,Kiribati,1994,30.000000
10936,SDN,Sudan,1962,1485.135000
1565,BRA,Brazil,2003,310809.997559
12047,TCD,Chad,1968,124.678000
5284,HPC,Heavily indebted poor countries (HIPC),2004,97029.998787
9779,PAN,Panama,1969,2053.520000
208,AFW,Africa Western and Central,1991,111479.470508


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13953 entries, 0 to 13952
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  13953 non-null  object 
 1   country_name  13953 non-null  object 
 2   year          13953 non-null  int64  
 3   value         13953 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 436.2+ KB
None
